# Cell Profiler - Per Patient
Nucleus rollup per patient.

In [1]:
import numpy as np
import pandas as pd
import os
import csv
from datetime import datetime
print(datetime.now())

2022-08-09 12:27:37.991956


In [2]:
BASE_PATH='/home/jrm/Adjeroh/Glioma/August_Run/CellProfilerPerPatient/'
NUCLEUS_INPUT='Process100_Nucleus.csv'
NUCLEUS_OUTPUT='NucleusRollupToPatient.csv'
IMAGE_COL='ImageNumber'

In [3]:
def save_patient(patient,one_row_as_dict):
    if patient is not None:
        filename = BASE_PATH + patient + '/' + NUCLEUS_OUTPUT
        print('Save',filename)
        df = pd.DataFrame(one_row_as_dict)
        df = df.replace('nan',np.nan)
        df.to_csv(filename,index=False) 

In [4]:
def drop_columns(df):
    cols = df.columns
    bad_cols=[c for c in cols if c in
              ['Children_Cells_Count','Number_Object_Number','ImageNumber','ObjectNumber']]
    loc_cols=[c for c in cols 
              if c.startswith('Location_')
              or c.startswith('AreaShape_BoundingBoxM')
              or c.startswith('AreaShape_Center')]
    df = df.drop(columns=bad_cols)
    df = df.drop(columns=loc_cols)
    return df

In [8]:
print(datetime.now())
for patient in os.listdir(BASE_PATH):
    if patient.startswith('TCGA-'):
        print(datetime.now())
        print('Processing',patient)
        filename = BASE_PATH + patient +'/' + NUCLEUS_INPUT
        bigdf = pd.read_csv(filename)
        bigdf = drop_columns(bigdf)

2022-08-09 12:41:54.718672
2022-08-09 12:41:54.719496
Processing TCGA-HT-7483-01Z-00
2022-08-09 12:41:56.588770
Processing TCGA-02-0025-01Z-00
2022-08-09 12:41:58.084683
Processing TCGA-DH-A66B-01Z-00
2022-08-09 12:41:58.562441
Processing TCGA-08-0517-01Z-00
2022-08-09 12:41:59.338306
Processing TCGA-26-5139-01Z-00
2022-08-09 12:42:00.811898
Processing TCGA-S9-A6WE-01Z-00
2022-08-09 12:42:01.131699
Processing TCGA-S9-A6U1-01Z-00
2022-08-09 12:42:01.348332
Processing TCGA-DH-A66G-01Z-00
2022-08-09 12:42:01.957506
Processing TCGA-S9-A7IZ-01Z-00
2022-08-09 12:42:02.274307
Processing TCGA-15-1446-01Z-00
2022-08-09 12:42:04.307597
Processing TCGA-S9-A6U9-01Z-00
2022-08-09 12:42:04.682326
Processing TCGA-02-0033-01Z-00
2022-08-09 12:42:05.510815
Processing TCGA-DH-A669-01Z-00
2022-08-09 12:42:07.034938
Processing TCGA-DB-5270-01Z-00
2022-08-09 12:42:07.665847
Processing TCGA-14-0786-01Z-00
2022-08-09 12:42:09.496386
Processing TCGA-HT-7676-01Z-00
2022-08-09 12:42:09.892628
Processing TCGA-HT

In [9]:
print(datetime.now())


2022-08-09 12:44:14.894286
